In [1]:
!pip install vosk
!pip install pipwin
!pip install pyaudio


In [2]:
import pyaudio
import wave
import json
import sys
from vosk import Model, KaldiRecognizer

# 1. Cargar el modelo de Vosk
model_path = r"vosk-model-es-0.42"
Model = Model(model_path)

# 2. Configurar PyAudio
p = pyaudio.PyAudio()

# Parám. de audio: 16 kHz, mono, 16 bits
RATE = 16000
CHANNELS = 1
FORMAT = pyaudio.paInt16
CHUNK = 4096  # tamaño de bloque

# 3. Abrir stream de audio desde el micrófono
stream = p.open(
    format=FORMAT,
    channels=CHANNELS,
    rate=RATE,
    input=True,
    frames_per_buffer=CHUNK
)

print("Micrófono abierto, habla algo... (Ctrl+C para salir)")

# Crear el reconocedor con la misma frecuencia de muestreo
rec = KaldiRecognizer(Model, RATE)

try:
    while True:
        # 4. Leer del micrófono en bloques
        data = stream.read(CHUNK, exception_on_overflow=False)
        
        # 5. Al alimentar el audio al reconocedor, Vosk determina si hay un resultado completo
        if rec.AcceptWaveform(data):
            # AcceptWaveform => frase terminada
            result_json = rec.Result()
            result_dict = json.loads(result_json)
            text = result_dict.get("text", "")
            print("** Frase reconocida:", text)
        else:
            # Caso contrario => partial result
            partial_json = rec.PartialResult()
            partial_dict = json.loads(partial_json)
            partial_text = partial_dict.get("partial", "")
            # Si quieres ver el texto parcial mientras hablas, descomenta:
            # print("...partial...", partial_text)
except KeyboardInterrupt:
    print("Saliendo...")

# 6. Cerrar y limpiar
stream.stop_stream()
stream.close()
p.terminate()


Micrófono abierto, habla algo... (Ctrl+C para salir)
** Frase reconocida: 
** Frase reconocida: hola cómo estás
** Frase reconocida: 
** Frase reconocida: actívate
Saliendo...


In [4]:
import pyaudio
import wave
import json
import sys
from vosk import Model as VoskModel, KaldiRecognizer

import torch
from PIL import Image
from transformers import VisionEncoderDecoderModel, AutoTokenizer, ViTImageProcessor

#####################
# 1. VOSK: CARGAR MODELO
#####################
def load_vosk_model(model_path=r"vosk-model-es-0.42", rate=16000):
    """
    Carga el modelo de Vosk y configura PyAudio para reconocer en vivo.
    """
    print(f"Cargando modelo Vosk desde: {model_path}")
    vosk_model = VoskModel(model_path)

    p = pyaudio.PyAudio()
    # Ajustes de audio: 16 kHz, mono, 16 bits
    RATE = rate
    CHANNELS = 1
    FORMAT = pyaudio.paInt16
    CHUNK = 4096

    stream = p.open(
        format=FORMAT,
        channels=CHANNELS,
        rate=RATE,
        input=True,
        frames_per_buffer=CHUNK
    )

    recognizer = KaldiRecognizer(vosk_model, RATE)
    return p, stream, recognizer


#####################
# 2. IMAGE CAPTIONING: CARGAR Y DESCRIBIR
#####################
def load_captioning_model(model_path="ViTGP", device="cpu"):
    print(f"Cargando modelo de Image Captioning desde: {model_path}")
    model = VisionEncoderDecoderModel.from_pretrained(model_path)
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    feature_extractor = ViTImageProcessor.from_pretrained(model_path)

    model.to(device)
    model.eval()
    return model, tokenizer, feature_extractor

def generate_caption(
    image_path,
    model,
    tokenizer,
    feature_extractor,
    device="cpu",
    max_length=30,
    num_beams=4
):
    img = Image.open(image_path).convert("RGB")
    pixel_values = feature_extractor(images=img, return_tensors="pt").pixel_values.to(device)

    with torch.no_grad():
        output_ids = model.generate(
            pixel_values,
            max_length=max_length,
            num_beams=num_beams
        )
    caption = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    return caption


#####################
# 3. ESCUCHA POR MICRÓFONO Y PROCESA COMANDOS
#####################
def listen_microphone(stream, recognizer, model, tokenizer, feature_extractor, device="cpu"):
    """
    Lee audio del micrófono en un bucle.
    - Si Vosk reconoce la palabra 'describe', genera la caption.
    - Si reconoce 'salir', rompe el bucle.
    """
    print("Micrófono activado. Di 'describe' para generar descripción, o 'salir' para terminar.")
    try:
        while True:
            data = stream.read(4096, exception_on_overflow=False)
            if recognizer.AcceptWaveform(data):
                result_json = recognizer.Result()
                result_dict = json.loads(result_json)
                recognized_text = result_dict.get("text", "").strip().lower()

                if recognized_text:
                    print(f"Reconocido: {recognized_text}")

                    if recognized_text == "describe":
                        caption = generate_caption(
                            image_path="B601743B-02FF-4B4F-B065-3B95473124D6-1030x773.jpeg",
                            model=model,
                            tokenizer=tokenizer,
                            feature_extractor=feature_extractor,
                            device=device,
                            max_length=120,
                            num_beams=4
                        )
                        print("Generated caption:", caption)

                    elif recognized_text == "salir":
                        print("Saliendo de la escucha por micrófono...")
                        break

            # El else atiende partial results, si quieres examinarlos
            # else:
            #    partial = recognizer.PartialResult()
            #    ...
    except KeyboardInterrupt:
        print("Interrumpido con Ctrl+C")


def run_commands(p, stream, recognizer, model, tokenizer, feature_extractor, device="cpu"):
    """
    Inicia la escucha del micrófono y ejecuta comandos según lo reconocido.
    Al terminar, cierra PyAudio.
    """
    listen_microphone(stream, recognizer, model, tokenizer, feature_extractor, device)
    stream.stop_stream()
    stream.close()
    p.terminate()


#####################
# 4. PUNTO DE ENTRADA PRINCIPAL
#####################
if __name__ == "__main__":
    device = "cuda" if torch.cuda.is_available() else "cpu"

    # 1) Activar y cargar el modelo de Vosk por defecto
    p, stream, recognizer = load_vosk_model(model_path="vosk-model-es-0.42", rate=16000)
    print("Modelo Vosk cargado correctamente.")

    # 2) Activar y cargar el modelo de Captioning por defecto
    model, tokenizer, feature_extractor = load_captioning_model(
        model_path="ViTGP",  # Ajustar a tu ruta de fine-tuning
        device=device
    )
    print("Modelo de Image Captioning cargado correctamente.")

    # 3) Ejecutar la lógica de comandos por voz
    run_commands(p, stream, recognizer, model, tokenizer, feature_extractor, device=device)


Cargando modelo Vosk desde: vosk-model-es-0.42
Modelo Vosk cargado correctamente.
Cargando modelo de Image Captioning desde: ViTGP


Config of the encoder: <class 'transformers.models.vit.modeling_vit.ViTModel'> is overwritten by shared encoder config: ViTConfig {
  "architectures": [
    "ViTModel"
  ],
  "attention_probs_dropout_prob": 0.0,
  "encoder_stride": 16,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.0,
  "hidden_size": 768,
  "image_size": 224,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "model_type": "vit",
  "num_attention_heads": 12,
  "num_channels": 3,
  "num_hidden_layers": 12,
  "patch_size": 16,
  "qkv_bias": true,
  "torch_dtype": "float32",
  "transformers_version": "4.49.0"
}

Config of the decoder: <class 'transformers.models.gpt2.modeling_gpt2.GPT2LMHeadModel'> is overwritten by shared decoder config: GPT2Config {
  "activation_function": "gelu_new",
  "add_cross_attention": true,
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "decoder_start_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id"

Modelo de Image Captioning cargado correctamente.
Micrófono activado. Di 'describe' para generar descripción, o 'salir' para terminar.
Reconocido: describe
Generated caption: Está preparando masa para pan o pizza. En la mesa de enfrente hay harina esparcida, un bol grande vacío y una taza medidora, los cuales suelen utilizarse en la preparación de masa para hornear.
Reconocido: salir
Saliendo de la escucha por micrófono...
